# Connecting Microsoft SQL Server with Python

This script demonstrates how to connect to a Microsoft SQL Server database using Python.

We will execute a sample query, fetch the results, load them into a Pandas DataFrame, and save them as a CSV file.


## Step 1: Set up the Conda Environment

Ensure the following commands are executed in your `Anaconda Prompt` terminal:

```
y to not ask yes/no
conda create -n ms-sql-env python=3.10 ipykernel -y
conda activate ms-sql-env
conda install -c conda-forge pyodbc pandas -y
```

The kernel does not get appeared to me --I used depi_env

## Step 2: Import Necessary Libraries

Import `pyodbc` for database connection and `pandas` for data manipulation.

In [1]:
import pyodbc
import pandas as pd

In [2]:
%pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


## Step 3: Define Connection Parameters

Replace the placeholder values with your actual SQL Server connection details.

In [3]:
server = r'Aya\SQLEXPRESS04'          # Replace with your server name
database = 'foe_db'                   # Replace with your database name
driver = '{ODBC Driver 17 for SQL Server}'  # Ensure this matches your installed driver from settings--programs
username = 'YourUsername'             # Replace with your SQL Server username (if needed)
password = 'YourPassword'             # Replace with your SQL Server password (if needed)

## Step 4: Test the Connection

Check if the connection with your DBMS is working correctly.

In [4]:
try:
    # Use Trusted Connection for Windows Authentication
    connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
    
    # For SQL Authentication, use the following line instead:
    # connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};'
    
    # Establish the connection
    connection = pyodbc.connect(connection_string)
    print("Connection is Successful!")

except pyodbc.Error as e:
    print("Error in connection:", e)


Connection is Successful!


## Step 5: Execute a Sample Query

Fetch data from the `Students` table (replace with your desired table or query).

In [18]:
# insert_query = """INSERT INTO Students (FirstName, LastName, Email)
#             VALUES ('Seif', 'Sameh', 'Seif_sameh@yahoo.com');
#         """
# cursor = connection.cursor()
# cursor.execute(insert_query)
# connection.commit()

In [5]:
# Replace 'Students' with your desired table
query = "SELECT TOP 10 * FROM Students"
# query = """
#     SELECT AVG(DATEDIFF(YEAR, BirthDate, GETDATE())) AS AverageAge
#     FROM Students
# """

# Load query results into a DataFrame
df = pd.read_sql(query, connection)
print(df)

Empty DataFrame
Columns: [StudentID, FirstName, LastName, BirthDate, Gender, Email]
Index: []


C:\Users\AYA\AppData\Local\Temp\ipykernel_25916\1929550570.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


## Step 6: Save Results to a CSV File

Save the DataFrame to a CSV file for further analysis.

In [5]:
# Replace with your desired file name
output_file = "students_table_updated.csv"
df.to_csv(output_file, index=False)
print(f"Query results save to {output_file}")

Query results save to students_table_updated.csv


## Step 7: Execute a Stored Procedure

Using pyodbc, Python executes the stored procedure `GetStudentsByCourseID`.

In [6]:
# Define the procedure name and parameters  
procedure_name = "GetStudentByCourseID"
course_id = 4

# Execute the stored procedure
cursor = connection.cursor()
cursor.execute(f"EXEC {procedure_name} @CourseID = ?", course_id) ##  EXEC GetStudentByCourseID @CourseID = 4

# Fetch all results into a pandas DataFrame
print(cursor.description)
print('-' * 40)
columns = []
# cursor.description is a list of what is contained in the table (column names, and its values)
for column in cursor.description:
    columns.append(column[0])
##columns = [column[0] for column in cursor.description]
results = cursor.fetchall()
print(results)
print('-' * 40)
df2 = pd.DataFrame.from_records(results, columns=columns)

# Display the DataFrame
print(df2)


# Optionally save results to a CSV file
df2.to_csv("students_by_course.csv", index=False)


(('StudentID', <class 'int'>, None, 10, 10, 0, False), ('FirstName', <class 'str'>, None, 50, 50, 0, True), ('LastName', <class 'str'>, None, 50, 50, 0, True), ('CourseName', <class 'str'>, None, 100, 100, 0, True), ('EnrollmentDate', <class 'datetime.date'>, None, 10, 10, 0, True))
----------------------------------------
[(4, 'Ali', 'Khan', 'Artificial Intelligence', datetime.date(2024, 9, 4))]
----------------------------------------
   StudentID FirstName LastName               CourseName EnrollmentDate
0          4       Ali     Khan  Artificial Intelligence     2024-09-04


## Step 8: Close the Database Connection

Always close the connection after use.

In [7]:
connection.close()
print("Connection Closed")

Connection Closed
